<a href="https://colab.research.google.com/github/CourantEnCourant/rag_course/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

**Charger un dataset depuis le Hub Hugging Face**

In [ ]:
from datasets import load_dataset


# Charger un dataset complet
squad_dataset = load_dataset("squad")

# Charger une configuration spécifique d'un dataset
dataset = load_dataset("squad_v2")

# Charger uniquement certains splits (train, validation, test)
train_dataset = load_dataset("squad", split="train")
validation_dataset = load_dataset("squad", split="validation")

# Charger un dataset hébergé par un utilisateur spécifique
dataset = load_dataset("stanfordnlp/imdb")

**Charger des datasets depuis des fichiers locaux**

In [ ]:
# Charger depuis un fichier CSV
dataset = load_dataset("csv", data_files="mon_fichier.csv")

# Charger plusieurs fichiers en un seul dataset
dataset = load_dataset("csv", data_files=["fichier1.csv", "fichier2.csv"])

# Spécifier des fichiers différents pour différents splits
dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv"})

# Autres formats supportés
json_dataset = load_dataset("json", data_files="corpus.json", field='data')
text_dataset = load_dataset("text", data_files="corpus.txt")
parquet_dataset = load_dataset("parquet", data_files="donnees.parquet")

In [ ]:
# Afficher les informations générales
print(dataset)

**Explorer la structure des datasets**

In [ ]:
# Voir les features (colonnes) du dataset
print(dataset["train"].features)

# A VOUS: Afficher quelques exemples

# A VOUS: Accéder à une colonne spécifique: quelle est la question 200 du dataset? Et la réponse?


# Obtenir des statistiques sur la taille du dataset
print(f"Nombre d'exemples dans l'ensemble d'entraînement : {len(dataset['train'])}")
print(f"Nombre d'exemples dans l'ensemble de validation : {len(dataset['validation'])}")

**Manipulation des datasets**

La bibliothèque datasets offre de nombreuses fonctions pour manipuler vos datasets.

**Filtrage et sélection**

In [ ]:
# Filtrer le dataset selon une condition

long_questions = dataset["train"].filter(lambda example: len(example["question"]) > 20)
print(f"Nombre de questions longues : {len(long_questions)}")


In [ ]:
# Sélectionner un sous-ensemble aléatoire
sample = dataset["train"].shuffle(seed=42).select(range(1000))

In [ ]:
# Filtrer avec plusieurs conditions
filtered = dataset["train"].filter(
    lambda example: len(example["question"]) > 10 and "why" in example["question"].lower()
)

**Mapping et transformation**

In [ ]:
# Appliquer une fonction à chaque exemple
def ajouter_longueur(example):
    example["question_length"] = len(example["question"])
    return example

dataset_with_lengths = dataset["train"].map(ajouter_longueur)

In [ ]:
# Appliquer des transformations sur plusieurs processus pour accélérer
transformed_dataset = dataset["train"].map(
    ajouter_longueur,
    num_proc=4,  # Utiliser 4 processus
    desc="Ajout des longueurs de questions"  # Description pour la barre de progression
)

**Tri et regroupement**

In [ ]:
# Trier le dataset
sorted_dataset = transformed_dataset.sort("question_length")

In [ ]:
# Grouper par le premier mot:

# A VOUS: Ajoutez la colonne "first_word" et afficher les 10 premiers mots les plus fréquents dans les questions



**Traitement et transformation**

In [ ]:
from transformers import AutoTokenizer

# A VOUS: tokénisez le dataset

In [ ]:
# Définir le format pour PyTorch, TensorFlow ou JAX
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids"])

In [ ]:
#Visualisez le premier exemple tokenizé:



**Sauvegarde et partage**

In [ ]:
# Sauvegarder un dataset sur disque
dataset["train"].save_to_disk("./mon_dataset_train")

# Charger depuis le disque
from datasets import load_from_disk
loaded_dataset = load_from_disk("./mon_dataset_train")

**Partager sur le Hub Hugging Face**

In [ ]:
# Pousser votre dataset sur le Hub
dataset.push_to_hub("votre-username/nom-du-dataset")


**Création d'un DataLoader pour l'entraînement**

In [ ]:
from torch.utils.data import DataLoader

# Convertir le dataset en format PyTorch
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids"])

# Créer un DataLoader
train_dataloader = DataLoader(
    tokenized_dataset,
    batch_size=16,
    shuffle=True
)

# Utilisation dans une boucle d'entraînement
for batch in train_dataloader:
    # Votre code d'entraînement ici
    print(batch["input_ids"].shape)
    break

**Créer votre propre dataset**

In [ ]:
from datasets import Dataset, Features, Value, ClassLabel

# Données d'exemple
data = {
    "text": ["J'adore ce film", "Ce film était terrible", "Film moyen"],
    "label": [1, 0, 1]
}

# Définir les features
features = Features({
    "text": Value("string"),
    "label": ClassLabel(names=["négatif", "positif"])
})

# Créer le dataset
my_dataset = Dataset.from_dict(data, features=features)
print(my_dataset)

**Streaming pour les grands datasets**

In [ ]:
# Charger un dataset en mode streaming
streaming_dataset = load_dataset("c4", "en", streaming=True)

# Itérer sur le dataset sans le charger entièrement en mémoire
for i, example in enumerate(streaming_dataset["train"]):
    print(example["text"][:100])
    if i >= 5:  # Afficher seulement les 5 premiers exemples
        break

**Exercices pratiques**

1. Exploration de dataset : Chargez le dataset SQUAD et identifiez les 10 mots les plus fréquents dans les questions.

2. Transformation et filtrage : Créez un sous-ensemble du dataset IMDB contenant uniquement les critiques positives de plus de 200 mots.

3. Création d'un dataset

In [ ]:
# A VOUS: Préparez deux versions de votre propre dataset pour un RAG à partir du dataset Wikipedia.dumps (https://huggingface.co/datasets/legacy-datasets/wikipedia):
# 1. Une version monolingue (anglais)
# 2. Une version trilingue (langues à votre choix)

# NB: ne prenez pas le corpus entier, il est trop grand.
# Limitez le volume du corpus et choisissez les textes qui appartiennent à un seul domaine (science, physique, linguistique etc.)
# Pour le dataset multilingue, rajoutez le label de langue